In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import warnings
np.random.seed(123)
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
# reading train and test file

train = pd.read_csv("/kaggle/input/jantahack-recommender-systems/train.csv")
test = pd.read_csv("/kaggle/input/jantahack-recommender-systems/test.csv")

In [ ]:
# convert the train in the long format to wide format

wide_train = train.pivot_table(index = "user_id", columns="challenge_sequence", values="challenge", aggfunc= lambda x : x).reset_index()

In [ ]:
# dropping the user_id, since we won't be needing those for our co-occurrence matrix

wide_train.drop(["user_id"], axis =1, inplace = True)

In [ ]:
# convert each row for a user into a string

rows = []
for index, row in wide_train.iterrows():
    r = " ".join(row.map(str)).lower()
    rows.append(r)

In [ ]:
# converting test to wide format

wide_test = test.pivot_table(index = "user_id", columns="challenge_sequence", values="challenge", aggfunc= lambda x : x).reset_index()

In [ ]:
wide_test.shape

In [ ]:
# saving test user_id for future use

test_ids = wide_test['user_id']

In [ ]:
# dropping user_id from wide test

wide_test.drop(["user_id"], axis =1, inplace = True)

In [ ]:
for index, row in wide_test.iterrows():
    r = " ".join(row.map(str)).lower()
    rows.append(r)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from fastai.text import *

In [ ]:
from pathlib import Path

In [ ]:
path = Path('/kaggle/working/')

Data in fast.ai is taken using TextLMDataBunch. This is very similar to ImageGenerator in Keras, where the path, labels, etc. are provided and the method prepares Train, Test and Validation data depending on the task at hand!

In [ ]:
pd.DataFrame(rows).to_csv('/kaggle/working/new_corpus.csv', index=False)

## Data Bunch for Language Model

In [ ]:
# creating a corpus
thefile = open("corpus.txt","w")

In [ ]:
for element in rows:
    thefile.write("%s\n"%element)

In [ ]:
thefile.close()

In [ ]:
%%time
data_lm = TextLMDataBunch.from_csv(path,'new_corpus.csv', text_cols = 0, include_bos = False)

In [ ]:
data_lm

In [ ]:
bs=16

In [ ]:
#torch.cuda.set_device(1)
#torch.cuda.set_device(0)

### Step 1. Training a Language Model

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.callbacks.append(ShowGraph(learn))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
learn.fit_one_cycle(4, 4e-2)

### Discrimiative Fine-Tuning

*learn.unfreeze()* makes all the layers of AWD_LSTM trainable. We can set a training rate using slice() function, which trains the last layer at 1e-02, while groups (of layers) in between would have geometrically reducing learning rates.

### Slated Triangular Learning Rates

This can achieved simply by using fit_one_cycle() method in fast.ai

### Gradual Unfreezing

Though I've not experimented with this here, the idea is pretty simple. In the start we keep the initial layers of model as un-trainable, and then we slowly unfreeze earlier layers, as we keep on training. I'll cover this in detail in next post

In [ ]:
learn.freeze_to(-3)

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn.fit_one_cycle(10, slice(1e-4, 1e-2))

In [ ]:
learn.fit_one_cycle(10, slice(1e-4, 1e-3))

In [ ]:
learn.model_dir = Path('/kaggle/working/')
learn.save(file = Path('language_model'))
learn.save_encoder(Path('language_model_encoder'))

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.model_dir = Path('/kaggle/input/modelencoder/')
learn.load_encoder('language_model_encoder (3)')

In [ ]:
rows[2]

In [ ]:
learn.predict(rows[2], n_words=6).upper()

In [ ]:
test_rows = []
for index, row in wide_test.iterrows():
    r = " ".join(row.map(str)).lower()
    test_rows.append(r)

In [ ]:
%%time
text = test_rows[2]
learn.predict(text, n_words = 1, temperature = 0.2)

In [ ]:
def get_pred(text):
    result = learn.predict(text, n_words = 3, temperature = 0.2)
    all_words = result.upper().split(" ")
    #print(result)
    outputs = list(all_words[10:13])
    return outputs

In [ ]:
def get_pred2(text, num_pred):
    
    start = text
    initial_words = text.split(" ")
    current_words = initial_words
    
    for i in range(0, num_pred):
        output = learn.predict(start, n_words = 1, temperature = 0.2)
        all_words = output.split(" ")
        count = 0
        while(all_words[-1] in current_words):
            count = count+1
            output = learn.predict(start, n_words = 1, temperature = 0.2)
            all_words = output.split(" ")
            if(count > 2):
                break;
            #print(count)
        start = output
        current_words = all_words
    
    #print(output)
    pred = list(output.upper().split(" ")[10:13])
    return pred

In [ ]:
test_rows[2].upper()

In [ ]:
%%time
get_pred(test_rows[2])

In [ ]:
%%time
get_pred2(test_rows[2], 3)

In [ ]:
test_ids

In [ ]:
preds_fastai = []
for i in range(19500,39732):
    print(i)
    preds = get_pred2(test_rows[i], 3)
    preds_fastai.append(preds)

In [ ]:
test_ids[19500]

In [ ]:
largest_3 = pd.DataFrame(preds_fastai)
largest_3['user_id'] = test_ids
largest_3_long = pd.melt(largest_3,id_vars="user_id",var_name="sequence", value_name="challenge" )
largest_3_long['sequence'] = largest_3_long['sequence'].map({0:'11',1:'12',2:"13"})
largest_3_long['user_sequence'] = largest_3_long['user_id'].map(str)+"_"+largest_3_long['sequence'].map(str)
largest_3_long.challenge[largest_3_long.challenge == ""] = "CI25555"
largest_3_long[['user_sequence','challenge']].to_csv("submission_lstm3.csv", index = False)